In [1]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import os
import json
from cloudpickle import dump
from pathlib import Path
import numpy as np
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import Client
import daal4py as d4p

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from typing import IO, AnyStr, Union, List, Optional, Tuple

gbt_params = {    
    'fptype':                       'float',
    'maxTreeDepth':                 3,
    'minSplitLoss':                 0.1,
    'shrinkage':                    0.1,
    'observationsPerTreeFraction':  1,
    'lambda_':                      1,
    'maxBins':                      256,
    'featuresPerNode':              0,
    'minBinSize':                   5,
    'minObservationsInLeafNode':    1,
    'nClasses':                     2
}

categories         = ['UniqueCarrier', 'Origin', 'Dest']
target_path        = '/User/repos/demos/dask/artifacts'


In [2]:
    # temporary hack
X = pd.read_parquet('/User/repos/demos/dask/dataset/partitions')

In [3]:
X.head()

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,ArrDelay,Origin,Dest,Distance
0,2003,12,6,6,1420.0,WN,23.0,STL,MDW,251.0
1,2003,12,8,1,1420.0,WN,-8.0,STL,MDW,251.0
2,2003,12,4,4,1420.0,WN,-7.0,STL,MDW,251.0
3,2003,12,2,2,1420.0,WN,-15.0,STL,MDW,251.0
4,2003,12,17,3,1420.0,WN,0.0,STL,MDW,251.0


In [4]:
y = X.pop('ArrDelay')
X['CRSDepTime'] = X['CRSDepTime'].clip(upper=2399)
y = (y.fillna(16) > 15)

In [5]:
le = LabelEncoder()
le = le.fit(y)
label_encoded_y = le.transform(y)

In [6]:
for cat in categories:
    X[cat] = le.fit_transform(X[cat])
    # loses this in the transform?
    X[cat] = X[cat].astype('category')
    fp = os.path.join(target_path, cat+'-encoding.pkl')
    # save this encoder
    dump(le, open(fp, 'wb'))

In [7]:
xtrain, xtest, ytrain, ytest = \
    train_test_split(X,
                     label_encoded_y,
                     test_size=0.98,
                     random_state=1)

    # One-hot encoding - todo

In [8]:
xtrain

,Year,Month,DayofMonth,DayOfWeek,CRSDepTime,UniqueCarrier,Origin,Dest,Distance
78098561,2008,4,18,5,1425.0,1,89,39,631.0
24456353,1991,2,26,2,1745.0,7,179,333,451.0
21958586,1992,8,12,3,1245.0,5,146,181,2556.0
122760634,1990,5,18,5,1625.0,25,69,187,544.0
21326301,1988,2,10,3,800.0,24,239,187,733.0
...,...,...,...,...,...,...,...,...,...
491263,1998,3,14,6,635.0,7,136,20,153.0
112989256,1990,11,11,7,2150.0,25,249,258,267.0
6762380,1995,10,7,6,1850.0,25,249,282,257.0
122126571,1994,8,1,1,110.0,7,115,248,42.0


In [20]:
xtest.shape

(121064270, 9)

In [21]:
ytrain.shape

(2470699,)

In [16]:
clf = d4p.gbt_classification_training(**gbt_params)
dir(clf)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_compute',
 'compute']

In [17]:
clf.compute?

Docstring:
gbt_classification_training.compute(data, labels, weights)
Do the actual computation on provided input data.

:param data_or_file data:  Training data set 
:param data_or_file labels:  Labels of the training data set 
:param data_or_file weights: [optional, default: None]  Optional. Weights of the observations in the training data set 
:rtype: gbt_classification_training_result
Type:      builtin_function_or_method


In [ ]:
clf_result = clf.compute(xtrain, ytrain)

In [ ]:
filepath = os.path.join(target_path, name)
dump(clf, open(filepath, 'wb'))
context.log_artifact(key, target_path=filepath)

filepath = os.path.join(target_path, 'results-' + name)
dump(clf_result, open(filepath, 'wb'))
context.log_artifact(key, target_path=filepath)